In [ ]:
import requests
import re
import os

from PIL import Image
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
from urllib.parse import urljoin
from urllib.parse import urlparse
from tldextract import extract
from pprint import pprint

url = 'http://aiacademy.tw/'

fu = UserAgent()
headers = {'User-Agent': fu.random}
resp = requests.get(url, headers=headers)
soup = BeautifulSoup(resp.text, 'lxml')

In [ ]:
def invalid_href(url):
    check_anchor = re.match('.*#.*', url)
    check_protocol = re.match('[^https|http].*', urlparse(url).scheme)
    check_js = re.match('javascript.*', url)
    return any([check_anchor, check_protocol, check_js])

def inner_href(url, domain):
    return extract(url).domain == domain

In [ ]:
wait_list = [url]
view_list = []

domain = extract(url).domain

while wait_list:
    link = wait_list.pop()
    if link in view_list:
        continue
        
    if incalid_href(link):
        continue
        
    if not inner_href(link, domain):
        continue
        
    print(link)
    view_list.append(link)
    
    page_resp = requests.get(link, headers=headers)
    page_soup = BeautifulSoup(page_resp.text, 'lxml')
    
    links = page_soup.find_all('a')
    links = [link['href'] for link in links if link.has_attr('href')]
    links = [urljoin(page_resp.url, link) for link in links]
    links = list(filter(lambda x: x not in view_list, links))
    
    wait_list += links
    wait_list = list(set(wait_list))

print('view list')
pprint(view_list)
    
    
    
    
    